In [ ]:
import requests

In [ ]:
appcode = 'Your AppCode here'

In [ ]:
areaid = "101291401"
month = "201805"

In [ ]:
url = 'https://weather01.market.alicloudapi.com/weatherhistory'
payload = {'areaid': areaid, 'month': month}
headers = {'Authorization': 'APPCODE {}'.format(appcode)}

In [ ]:
r = requests.get(url, params=payload, headers=headers)

In [ ]:
r

In [ ]:
r.content

In [ ]:
import json

In [ ]:
content_json = json.loads(r.content)

In [ ]:
content_json

In [ ]:
content_json['showapi_res_body']['list']

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(content_json['showapi_res_body']['list'])

In [ ]:
df

In [ ]:
def get_df(areaid, areaname_dict, month, appcode):

    url = 'https://weather01.market.alicloudapi.com/weatherhistory'
    payload = {'areaid': areaid, 'month': month}
    headers = {'Authorization': 'APPCODE {}'.format(appcode)}

    r = requests.get(url, params=payload, headers=headers)

    content_json = json.loads(r.content)

    df = pd.DataFrame(content_json['showapi_res_body']['list'])
    df['areaname'] = areaname_dict[areaid]

    return df

In [ ]:
def get_dfs(areaname_dict, months, appcode):
    dfs = []
    for areaid in areaname_dict:
        dfs_times = []
        for month in months:
            temp_df = get_df(areaid, areaname_dict, month, appcode)
            dfs_times.append(temp_df)
        area_df = pd.concat(dfs_times)
        dfs.append(area_df)
    return dfs

In [ ]:
areaname_dict = {"101291401":"丽江"}
months = ["201805"]

In [ ]:
dfs = get_dfs(areaname_dict, months, appcode)

In [ ]:
dfs

In [ ]:
dfs[0]

In [ ]:
areaname_dict = {"101030100":"天津", "101020100":"上海", "101291401":"丽江"}

In [ ]:
months = ["201801", "201802", "201803", "201804", "201805"]

In [ ]:
dfs = get_dfs(areaname_dict, months, appcode)

In [ ]:
dfs

In [ ]:
df = pd.concat(dfs)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df.time = pd.to_datetime(df.time)

In [ ]:
df.aqi = pd.to_numeric(df.aqi)

In [ ]:
df.dtypes

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from plotnine import *
from mizani.breaks import date_breaks

In [ ]:
(ggplot(df, aes(x='time', y='aqi', color='factor(areaname)')) + geom_line() +
 scale_x_datetime(breaks=date_breaks('2 weeks')) +
 xlab('日期') + 
 theme_matplotlib() +
 theme(axis_text_x=element_text(rotation=45, hjust=1)) +
 theme(text=element_text(family='WenQuanYi Micro Hei'))
 )